In [10]:
from xgboost import XGBClassifier
from ml_pipeline import MLPipeline

In [11]:
# Check GPU availability
import subprocess
try:
    result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], 
                          capture_output=True, text=True, check=True)
    print("✅ GPU Detected:")
    print(result.stdout)
except:
    print("⚠️  Warning: nvidia-smi not available or no GPU detected")
    print("XGBoost will fallback to CPU if GPU is not available")

✅ GPU Detected:
NVIDIA GeForce RTX 5070 Ti, 16303 MiB



# XGBoost with Wavelet Transform on Fashion-MNIST (GPU Accelerated)

This notebook uses the MLPipeline class to train an XGBoost classifier with wavelet feature extraction.

**GPU Acceleration:** XGBoost is configured to use NVIDIA GPU (RTX 5070 Ti) for faster training via CUDA.

In [12]:
# Initialize pipeline with XGBoost model configured for GPU
model = XGBClassifier(
    random_state=42,
    device='cuda',  # Use NVIDIA GPU
    tree_method='hist',  # Histogram-based algorithm (GPU compatible)
    n_jobs=1  # GPU handles parallelism internally
)
pipeline = MLPipeline(model=model, wavelet='bior1.3')

# Load data with 1/60 subset ratio
pipeline.load_data(
    train_path='./data/fashion-mnist_train.csv',
    test_path='./data/fashion-mnist_test.csv',
    subset_ratio=1/60
)

╭─────────── 📊 Loading Data ───────────╮
│ Loading datasets...                   │
│ Train: ./data/fashion-mnist_train.csv │
│ Test: ./data/fashion-mnist_test.csv   │
╰───────────────────────────────────────╯

⚠️  Applying subset ratio: 1.67%

        Dataset Summary         
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃ Metric               ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│ Train samples        │ 1000  │
│ Test samples         │ 167   │
│ Features per sample  │ 784   │
│ Subset ratio applied │ 1.67% │
└──────────────────────┴───────┘

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ✅ Data loaded successfully!                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

MLPipeline(model=XGBClassifier, wavelet='bior1.3', trained=False)

# GridSearchCV for Hyperparameter Tuning

In [ ]:
# Define hyperparameter grid
param_grid = {
    'max_depth': [3, 5, 10, 15, 100, 500],
    'n_estimators': [300, 500, 1000, 10000, 15000, 20000],
}

# Fit model with GridSearchCV (cv=2 for faster execution)
pipeline.fit_with_gridsearch(
    param_grid=param_grid,
    cv=2,
    scoring='accuracy',
    use_wavelets=True,
    normalize=True,
    verbose=1
)

╭─ 🔍 GridSearchCV Hyperparameter Tuning ─╮
│ Model: XGBClassifier                    │
│ Cross-validation folds: 2               │
│ Scoring metric: accuracy                │
│ Use wavelets: Yes                       │
│ Normalize: Yes                          │
╰─────────────────────────────────────────╯

                     Parameter Grid                     
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Parameter    ┃ Values                                ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ max_depth    │ [3, 5, 10, 15, 100, 500]              │
│ n_estimators │ [300, 500, 1000, 10000, 15000, 20000] │
└──────────────┴───────────────────────────────────────┘

Preparing features...

Output()

Starting GridSearchCV with 2-fold cross-validation...

Fitting 2 folds for each of 36 candidates, totalling 72 fits


/home/valentin/Documents/Ynov/Projet1_ML/.venv/lib/python3.13/site-packages/xgboost/core.py:774: UserWarning: [10:43:11] WARNING: /workspace/src/common/error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)
/home/valentin/Documents/Ynov/Projet1_ML/.venv/lib/python3.13/site-packages/xgboost/core.py:774: UserWarning: [10:43:12] WARNING: /workspace/src/common/error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matche

# Model Evaluation

In [ ]:
# Make predictions on test set
pipeline.predict()

In [ ]:
# Display confusion matrix and classification report
pipeline.evaluate(show_plot=True, show_report=True)

In [ ]:
# Get accuracy score with verbose output
accuracy = pipeline.get_accuracy(verbose=True)